In [66]:
import z3
import hashlib

In [67]:
with open("ciphertext.txt") as f:
    ct = f.read().strip()
import ast
ct = ast.literal_eval(ct)


with open("flag_hash.txt") as f:
    plaintext_hash = f.read().strip()
    plaintext_hash,salt  = plaintext_hash.split(":")

wheel_len = 64


In [68]:
solver = z3.Solver()

wheel = [z3.Int(f"wheel_{i}") for i in range(wheel_len)]

for i in range(wheel_len):
    solver.add(z3.And(wheel[i] >= 0, wheel[i] <= 1))

In [69]:
# define three state for wheel bits:
# 0: unknown
# -X: known to be state A for group X.
# X: known to be state B for group X.
# after iterating every groups, we will have multiple groups, then we do brute force to find the key.
ct_ptr = 0
wheel_ptr = -1
while ct_ptr < len(ct):
    distance = ct[ct_ptr]

    # for example, we have a number 7 at the first ciphertext,
    # then we can know the key is either 0000001 or 1111110.

    position = wheel_ptr + distance

    # handle looping of the wheel.
    position_real = position % wheel_len

    for i in range(position-1, wheel_ptr, -1):
        # handle looping
        i_real = i % wheel_len
        print(f"wheel[{position_real}] != wheel[{i}]")
        solver.add(wheel[position_real] != wheel[i_real])
    wheel_ptr = position_real
    ct_ptr += 1

# print(wheel)

wheel[1] != wheel[0]
wheel[6] != wheel[5]
wheel[6] != wheel[4]
wheel[11] != wheel[10]
wheel[11] != wheel[9]
wheel[13] != wheel[12]
wheel[18] != wheel[17]
wheel[18] != wheel[16]
wheel[18] != wheel[15]
wheel[21] != wheel[20]
wheel[24] != wheel[23]
wheel[24] != wheel[22]
wheel[29] != wheel[28]
wheel[33] != wheel[32]
wheel[35] != wheel[34]
wheel[37] != wheel[36]
wheel[41] != wheel[40]
wheel[41] != wheel[39]
wheel[48] != wheel[47]
wheel[48] != wheel[46]
wheel[48] != wheel[45]
wheel[48] != wheel[44]
wheel[48] != wheel[43]
wheel[54] != wheel[53]
wheel[54] != wheel[52]
wheel[54] != wheel[51]
wheel[0] != wheel[63]
wheel[3] != wheel[2]
wheel[8] != wheel[7]
wheel[8] != wheel[6]
wheel[11] != wheel[10]
wheel[11] != wheel[9]
wheel[13] != wheel[12]
wheel[18] != wheel[17]
wheel[18] != wheel[16]
wheel[24] != wheel[23]
wheel[24] != wheel[22]
wheel[28] != wheel[27]
wheel[28] != wheel[26]
wheel[28] != wheel[25]
wheel[32] != wheel[31]
wheel[32] != wheel[30]
wheel[35] != wheel[34]
wheel[37] != wheel[36]
whe

In [70]:

from chal import K_Cessation as Homework,decode_ascii_with_random_msb

def try_decrypt(key):
    key = key
    h = Homework(key)
    flag0 = decode_ascii_with_random_msb(h.decrypt(ct))
    for flag in [flag0]:
        if hashlib.sha256((salt).encode() + flag).hexdigest() == plaintext_hash:
            print(key)
            print(flag)
            return True
    return False
count = 0
while solver.check() == z3.sat:
    count+=1
    m = solver.model()
    key = []
    for i in range(wheel_len):
        key.append(m[wheel[i]].as_long())
    if try_decrypt(key):
        print("found after",count,"tries")
        break
    block = []
    for d in m:
        c = d()
        block.append(c != m[d])
    solver.add(z3.Or(block))
else:
    print("not found after",count,"tries")


[1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0]
b'DoubleUmCtF[S33K1NG_tru7h-7h3_w1s3-f1nd_1n57e4d-17s_pr0f0und-4b5ence_n0w-g0_s0lv3-th3_3y3s-1n_N0ita]'
found after 3 tries
